# DARP Benchmark Instances

Este notebook tem como objetivo estudar e analisar os dados extraidos do [Scopus](https://www.scopus.com/) sobre os três artigos que primeiramente propuseram as instâncias de benchmark para o DARP e que são mencionadas por Ho et al. (2018).

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
plt.style.use('dark_background')
% matplotlib inline

In [2]:
# Importar dados dos arquivos, e concatena-los

arquivos = ["darp/cordeau&laporte(2003)/cite_cordeau&laporte(2003).csv",
            "darp/cordeau(2006)/cite_cordeau(2006).csv",
            "darp/ropke_etal(2007)/cite_ropke_etal(2007).csv",
            "darp/ropke&cordeau(2009)/cite_ropke&cordeau(2009).csv"]

colunas_relevantes = ['Authors', 'Title', 'Year', 'Source title', 'Author Keywords', 'Index Keywords',
                      'Abstract', 'Cited by', 'Document Type', 'Language of Original Document']

def importar_csv_com_fonte(arquivo):
    df = (pd.read_csv(arquivo, usecols=colunas_relevantes)
            .assign(CiteArticle = re.search("/cite_(.*).csv", arquivo).group(1))
    )
    return (df)


dados = (pd.concat(map(importar_csv_com_fonte, arquivos),)
           .reset_index(drop=True)
)

In [3]:
# Verificar se todos os dados foram corretamente importados
dados.groupby("CiteArticle")["Title"].count()

CiteArticle
cordeau&laporte(2003)    265
cordeau(2006)            192
ropke&cordeau(2009)      121
ropke_etal(2007)         125
Name: Title, dtype: int64

In [4]:
# Deletar dados duplicados
dados.drop_duplicates(subset=colunas_relevantes, inplace=True)

Com os dados importados, procura-se filtrar os artigos que tratem de problemas dinâmicos. Para isso, será selecionado todo o artigo que mencionar a palavra "Dynamic" no título, resumo ou lista de palavras-chave

In [5]:
dados_dinamicos = (dados.assign(AllKeywords = lambda x: x["Author Keywords"].str.cat(x["Index Keywords"], na_rep=""))
                        .assign(DynamicTitle = lambda x: x.Title.str.contains("Dynamic"),
                                DynamicKeywords = lambda x: x.AllKeywords.str.contains("Dynamic"),
                                DynamicAbstract = lambda x: x.Abstract.str.contains("Dynamic"))
                        .assign(DynamicAny = lambda x: (x.DynamicTitle | x.DynamicKeywords | x.DynamicAbstract))                        
)
dados_dinamicos = dados_dinamicos.loc[dados_dinamicos["DynamicAny"] == True,:]

In [6]:
dados_dinamicos.shape

(56, 16)

In [7]:
dados_dinamicos.groupby("Document Type")["Title"].count()

Document Type
Article             39
Book Chapter         1
Conference Paper    13
Review               2
Short Survey         1
Name: Title, dtype: int64

Com esse filtro aplicado, sobram 56 itens, dos quais 39 são artigos de revista, 13 são artigos de conferência. Lendo o resumo do livro, das revisões e da pesquisa, coclui-se que estes não são de interesse para análise, já quenão apresentam experimentos computacionais.

In [8]:
artigos_dinamicos = dados_dinamicos.loc[dados_dinamicos["Document Type"].isin(["Article", "Conference Paper"])]

In [9]:
artigos_dinamicos.groupby("Language of Original Document")["Title"].count()

Language of Original Document
English    51
French      1
Name: Title, dtype: int64

Lendo o resumo dos artigos restantes, foi possível determinar quais tratam de DARP, ou PDPTW dinâmicos e quais executam experimentos computacionais. Todos os artigos que não cumprem essas exigências foram deletados da análise.

In [10]:
indices_para_dropar = [53, 58, 62, 63, 64, 72, 73, 78, 93, 118, 128, 133, 140, 144, 148, 160, 185, 192,
                       205, 226, 238, 281, 313, 319, 327, 358, 387, 407, 417, 459, 522, 562, 567, 593,
                       597, 615, 636, 644, 700]

indices_para_dropar.sort()

In [11]:
artigos_realmente_dinamicos = artigos_dinamicos.drop(labels=indices_para_dropar)

Através da leitura das seções dos artigos que denotam as características dos experimentos, observa-se quais as instâncias usadas. A informação observada está inserida na coluna `"Instances"` no `DataFrame` `artigos_dinamicos`.

In [12]:
artigos_instancias = artigos_realmente_dinamicos.assign(Instance = False)
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Double-horizon based heuristics for"),
                      "Instance"] = "Self-made, Realistic"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Waiting strategies for the dynamic pickup and"),
                      "Instance"] = "Self-made, Realistic"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Dynamic transportation of patients in hospitals"),
                      "Instance"] = "Self-made, Real"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Metaheuristics for the dynamic stochastic di"),
                      "Instance"] = "Self-made, Realistic, Dynamization"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("On dynamic pickup and delivery vehicle"),
                      "Instance"] = "Self-made, Realistic"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Large scale realtime ridesharing with"),
                      "Instance"] = "Self-made, Real"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("A tabu search heuristic for the dynamic "),
                      "Instance"] = "Self-made, Real, Realistic"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("A hybrid tabu search and constraint programming algorithm"),
                      "Instance"] = "Cordeau(2006), Dynamization"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Checking the feasibility of dial-a-ride instances using c"),
                      "Instance"] = "Cordeau(2006), Dynamization"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("On dynamic demand responsive transport services with degree"),
                      "Instance"] = "Self-made"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Waiting strategies for the dynamic dial-a-ride problem"),
                      "Instance"] = "Self-made"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Maximizing the number of served requests in an online shared"),
                      "Instance"] = "Self-made"
artigos_instancias.loc[artigos_instancias["Title"].str.contains("Determination of robust solutions for the dynamic dial-a-ri"),
                      "Instance"] = "Self-made"

In [13]:
artigos_instancias.shape[0]

13

In [14]:
artigos_instancias.sort_values("Cited by", ascending=False)

,Authors,Title,Year,Source title,Cited by,Abstract,Author Keywords,Index Keywords,Language of Original Document,Document Type,CiteArticle,AllKeywords,DynamicTitle,DynamicKeywords,DynamicAbstract,DynamicAny,Instance
264,"Mitrović-Minić S., Krishnamurti R., Laporte G.",Double-horizon based heuristics for the dynami...,2004,Transportation Research Part B: Methodological,142.0,The dynamic Pickup and Delivery Problem with T...,Dynamic pickup and delivery problem with time ...,Costs; Heuristic methods; Industrial managemen...,English,Article,cordeau&laporte(2003),Dynamic pickup and delivery problem with time ...,False,True,False,True,"Self-made, Realistic"
263,"Mitrović-Minić S., Laporte G.",Waiting strategies for the dynamic pickup and ...,2004,Transportation Research Part B: Methodological,109.0,The dynamic pickup and delivery problem with t...,Dynamic pickup and delivery problem with time ...,Pickups; Problem solving; Courier companies; S...,English,Article,cordeau&laporte(2003),Dynamic pickup and delivery problem with time ...,False,True,False,True,"Self-made, Realistic"
196,"Beaudry A., Laporte G., Melo T., Nickel S.",Dynamic transportation of patients in hospitals,2010,OR Spectrum,72.0,This study analyzes and solves a patient trans...,Dial-a-ride; Dynamic mode; In-house hospital t...,NaN,English,Article,cordeau&laporte(2003),Dial-a-ride; Dynamic mode; In-house hospital t...,True,True,False,True,"Self-made, Real"
164,"Schilde M., Doerner K.F., Hartl R.F.",Metaheuristics for the dynamic stochastic dial...,2011,Computers and Operations Research,48.0,The problem of transporting patients or elderl...,Dial-a-ride problem; Dynamic problem; Metaheur...,Dial-a-ride problem; Dynamic problem; Metaheur...,English,Article,cordeau&laporte(2003),Dial-a-ride problem; Dynamic problem; Metaheur...,False,True,False,True,"Self-made, Realistic, Dynamization"
248,"Fabri A., Recht P.",On dynamic pickup and delivery vehicle routing...,2006,Transportation Research Part B: Methodological,46.0,"In 2001, Caramia and his coauthors introduced ...",Dial-a-ride; Dynamic; Pickup and delivery; Tim...,Algorithms; Dynamic programming; Fleet operati...,English,Article,cordeau&laporte(2003),Dial-a-ride; Dynamic; Pickup and delivery; Tim...,False,True,False,True,"Self-made, Realistic"
351,"Huang Y., Bastani F., Jin R., Wang X.S.",Large scale realtime ridesharing with service ...,2014,Proceedings of the VLDB Endowment,42.0,Urban traffic gridlock is a familiar scene. At...,NaN,Algorithms; Amphibious vehicles; Branch and bo...,English,Conference Paper,cordeau(2006),Algorithms; Amphibious vehicles; Branch and bo...,False,True,False,True,"Self-made, Real"
168,"Kergosien Y., Lenté C., Piton D., Billaut J.-C.",A tabu search heuristic for the dynamic transp...,2011,European Journal of Operational Research,28.0,The problem studied in this paper stems from a...,Health care; Real-time; Tabu search; Transport...,Adaptive memory; Central stations; Computation...,English,Article,cordeau&laporte(2003),Health care; Real-time; Tabu search; Transport...,False,True,False,True,"Self-made, Real, Realistic"
153,"Berbeglia G., Cordeau J.-F., Laporte G.",A hybrid tabu search and constraint programmin...,2012,INFORMS Journal on Computing,26.0,This paper introduces a hybrid algorithm for t...,Constraint programming; Dial-a-ride problem; D...,Constraint programming; Dial-a-ride problem; E...,English,Article,cordeau&laporte(2003),Constraint programming; Dial-a-ride problem; D...,False,True,False,True,"Cordeau(2006), Dynamization"
181,"Berbeglia G., Pesant G., Rousseau L.-M.",Checking the feasibility of dial-a-ride instan...,2011,Transportation Science,16.0,"In the dial-a-ride problem (DARP), a fleet of ...",Algorithms; Constraint programming; Dial-A-Rid...,Algorithms; Computer programming; Constraint t...,English,Article,cordeau&laporte(2003),Algorithms; Constraint programming; Dial-A-Rid...,False,True,False,True,"Cordeau(2006), Dynamization"
103,"Wong K.I., Han A.F., Yuen C.W.",On dynamic de

Com isso, analizamos os métodos de dinamização de cada um dos três artigos que usam deste artifício.

In [15]:
artigos_instancias[artigos_instancias["Instance"].str.contains("Dynamization")]

,Authors,Title,Year,Source title,Cited by,Abstract,Author Keywords,Index Keywords,Language of Original Document,Document Type,CiteArticle,AllKeywords,DynamicTitle,DynamicKeywords,DynamicAbstract,DynamicAny,Instance
153,"Berbeglia G., Cordeau J.-F., Laporte G.",A hybrid tabu search and constraint programmin...,2012,INFORMS Journal on Computing,26.0,This paper introduces a hybrid algorithm for t...,Constraint programming; Dial-a-ride problem; D...,Constraint programming; Dial-a-ride problem; E...,English,Article,cordeau&laporte(2003),Constraint programming; Dial-a-ride problem; D...,False,True,False,True,"Cordeau(2006), Dynamization"
164,"Schilde M., Doerner K.F., Hartl R.F.",Metaheuristics for the dynamic stochastic dial...,2011,Computers and Operations Research,48.0,The problem of transporting patients or elderl...,Dial-a-ride problem; Dynamic problem; Metaheur...,Dial-a-ride problem; Dynamic problem; Metaheur...,English,Article,cordeau&laporte(2003),Dial-a-ride problem; Dynamic problem; Metaheur...,False,True,False,True,"Self-made, Realistic, Dynamization"
181,"Berbeglia G., Pesant G., Rousseau L.-M.",Checking the feasibility of dial-a-ride instan...,2011,Transportation Science,16.0,"In the dial-a-ride problem (DARP), a fleet of ...",Algorithms; Constraint programming; Dial-A-Rid...,Algorithms; Computer programming; Constraint t...,English,Article,cordeau&laporte(2003),Algorithms; Constraint programming; Dial-A-Rid...,False,True,False,True,"Cordeau(2006), Dynamization"
